# **Kaggle Project**

# "Personal Key Indicators of Heart Disease"

---


- Cormet Dylan
- Lheureux Dylan
- Batard Robin
- Choiset Flore

---

**Quels sont les facteurs pouvant provoquer des attaques cardiaques ?**

## Première partie (rendu 1) : Analyse globale du dataset

In [ ]:
#Importation librairies
library(tidyverse) 
library(ggplot2)
library(ggthemes)
library(caTools)
library(caret)
library(forestmodel)
library(dplyr)
library(broom)
library(ggeffects)
library(yardstick)
library(rpart)
library(party)
library(randomForest)

Dans cette première partie nous allons tout d'abord améliorer le format du dataset. C'est à dire  transformer les strings en int.  Nous avons aussi créé deux nouvelles colonnes "AgeInf" et "AgeSup" à partir de la colonne AgeCategory qui définissait un intervalle à partir d'un string. 

In [ ]:
list.files(path = "../input")
#importation dataset
data <- read.csv("../input/personal-key-indicators-of-heart-disease/2022/heart_2022_with_nans.csv")
#sélection des colonnes
data <- subset(data, select = -c(State, HadAngina, HadCOPD, HadDepressiveDisorder, HadKidneyDisease, HadArthritis, DeafOrHardOfHearing, BlindOrVisionDifficulty, DifficultyConcentrating, DifficultyDressingBathing, DifficultyErrands, ChestScan, HighRiskLastYear, CovidPos, PhysicalHealthDays, MentalHealthDays, ECigaretteUsage, HIVTesting, FluVaxLast12, PneumoVaxEver, TetanusLast10Tdap, HeightInMeters, WeightInKilograms, LastCheckupTime, RemovedTeeth))
#On enleve les NA
data <- na.omit(data)
for (i in 1:length(data)){
data <- subset(data, data[i] !="")
    }
summary(data)
#On transforme les colonnes Yes/No en 1/0
convert_Yes_No <- function(column) {
  ifelse(column == "No", 0, ifelse(column == "Yes", 1, column))
}

# Appliquer la fonction à toutes les colonnes spécifiées
columns_to_convert <- c("PhysicalActivities", "HadHeartAttack",
                        "HadStroke", "HadAsthma", "HadSkinCancer", 
                        "HadDiabetes", "AlcoholDrinkers", "DifficultyWalking")

#On transforme "Male" en 0 et "Female" en 1 dans la colonne Sex
data[columns_to_convert] <- lapply(data[columns_to_convert], convert_Yes_No)
data$Sex <- ifelse(data$Sex == "Male", 0, 
                    ifelse(data$Sex == "Female", 1, data$Sex)) 
data$HadDiabetes[data$HadDiabetes == 'No, pre-diabetes or borderline diabetes'] <- 0
data$HadDiabetes[data$HadDiabetes == 'Yes, but only during pregnancy (female)'] <- 0

On affiche un résumé de notre dataset, on observe que l'on a 346165 lignes pour chaque variable. 

In [ ]:
# Création d'un vecteur avec les valeurs uniques de la colonne GeneralHealth
valeurs_uniques <- c("Poor","Fair","Good","Very good","Excellent")

# Création d'une correspondance entre les valeurs uniques et les chiffres
correspondance <- setNames(1:length(valeurs_uniques), valeurs_uniques)

# Remplacement des valeurs de la colonne par les chiffres correspondants
data$GeneralHealth_numerique <- correspondance[data$GeneralHealth]
valeurs_uniques


In [ ]:
#On transforme la colonne représentant les intervalles d'âges, trop dure à utiliser telle quelle
# Fonction pour extraire le premier nombre
extraire_premier_nombre <- function(phrase) {
  nombre <- as.numeric(sub("^Age (\\d+).*", "\\1", phrase))
  if (is.na(nombre)) {
    nombre <- NA
  }
  return(nombre)
}


# Fonction pour extraire le deuxième nombre
extraire_deuxieme_nombre <- function(phrase) {
  if (grepl("80 or older", phrase)) {
    nombre <- "100"
  } else {
    nombre <- gsub("^Age \\d+ to (\\d+)", "\\1", phrase)
  }
  return(nombre)
}

data$AgeInf <- sapply(data$AgeCategory, extraire_premier_nombre)
data$AgeSup <- sapply(data$AgeCategory, extraire_deuxieme_nombre)

In [ ]:
#Pour la colonne des fumeurs on simplifie en "A deja fumé" : 1 et "n'a jamais fumé" : 0
# Parcours de la colonne et remplacement des valeurs
data$SmokerStatus <- ifelse(data$SmokerStatus == "Never smoked", 0, 1)

In [ ]:
head(data)
# Conversion de toutes les colonnes qui avaient des Strings : 
#SmokerStatus : 0 si n'a jamais fumé et 1 si a déjà fumé
# Sex : 0 si homme 1 si femme
# GeneralHealth -> ajout de GeneralHealth_numerique : 1 si excellent, 2 si very good, 4 poor et 5 good
#AgeCategory : Creation de AgeInf et Agesup avec Ageinf la borne inf de l'intervalle 
# ex : AgeInf vaut 65 pour "Age 65 to 69" et AgeSup vaut 69. Si on a "80 or older" AgeSup vaut 100
# Pour toute autre colonne avec Yes/No : No vaut  0 et Yes vaut 1 

On affiche les premiers éléments de notre dataset pour vérifier que l'on a bien réussi à créer nos nouvelles variables. 

In [ ]:
mean(data$HadHeartAttack ==1)*100

On a d'abord calculé la probabilité moyenne de faire une attaque cardiaque ( de manière globale ). Cette probabilité est de 5.65%. On pourra par la suite s'en servir de référence afin de comparer si telle ou telle variable possède une influence sur le risque de faire une attaque cardiaque. 

In [ ]:
#Création du graphique représentant la Probabilité de faire une attaque cardiaque sachant qu'on est malade
# Calcul des proportions
proportions <- c(
  sum(data$HadHeartAttack == 1 & data$HadStroke == 1) / sum(data$HadStroke == 1) * 100,
  sum(data$HadHeartAttack == 1 & data$HadAsthma == 1) / sum(data$HadAsthma == 1) * 100,
  sum(data$HadHeartAttack == 1 & data$HadSkinCancer == 1) / sum(data$HadSkinCancer == 1) * 100,
  sum(data$HadHeartAttack == 1 & data$HadDiabetes == 1) / sum(data$HadDiabetes == 1) * 100,
  sum(data$HadHeartAttack == 1 & data$HadDiabetes == 0 & data$HadStroke == 0 & data$HadAsthma == 0 & data$HadSkinCancer == 0) / sum(data$HadDiabetes == 0 & data$HadStroke == 0 & data$HadAsthma == 0 & data$HadSkinCancer == 0) * 100
)

# Noms des variables
names <- c("Stroke", "Asthma", "Skin Cancer", "Diabetes","No disease")

# Création de l'histogramme
barplot(proportions, 
        names.arg = names, 
        xlab = "Maladie", 
        ylab = "Pourcentage",
        main = "Probabilité de faire une attaque cardiaque sachant qu'on est malade",
        ylim = c(0, max(proportions) * 1.2))  # Échelonnage de l'axe des y

# Ajout des pourcentages sur les barres
text(x = 1:length(proportions), 
     y = proportions, 
     labels = paste(round(proportions, 2), "%"), 
     pos = 3, 
     cex = 0.8, 
     col = "blue")

png("pourcentage_maladies.png", width = 800, height = 600)
dev.off()

On a d'abord calculé la probabilité moyenne de faire une attaque cardiaque ( de manière globale ). Cette probabilité est de 5.65%. On pourra par la suite s'en servir de référence afin de comparer si telle ou telle variable possède une influence sur le risque de faire une attaque cardiaque. 

In [ ]:
#Création du graphique représentant la Distribution des heures de sommeil : s'il y a eu une attaque cardiaque ou non
# Séparer les données en deux sous-ensembles pour chaque valeur de HadHeartAttack
data_heart_attack_0 <- data[data$HadHeartAttack == 0, ]
data_heart_attack_1 <- data[data$HadHeartAttack == 1, ]



# Créer un graphique avec superposition de PMF pour chaque groupe, en normalisant les fréquences
ggplot() +
  geom_bar(data = data_heart_attack_1, aes(x = SleepHours, y = after_stat(count/sum(count)), fill = "A eu une attaque cardiaque"), alpha = 0.5, position = "identity") +
  geom_bar(data = data_heart_attack_0, aes(x = SleepHours, y = after_stat(count/sum(count)), fill = "N'a pas eu d'attaque cardiaque"), alpha = 0.5, position = "identity") +
  labs(x = "Heures de sommeil", y = "Fréquence", fill = "Group") +
  ggtitle("Distribution des heures de sommeil : s'il y a eu une attaque cardiaque ou non") +
  scale_fill_manual(values = c("A eu une attaque cardiaque" = "red", "N'a pas eu d'attaque cardiaque" = "blue"),
                     labels = c("A eu une attaque cardiaque", "N'a pas eu d'attaque cardiaque")) +
  theme_minimal() +
  xlim(0, 15)  # Fixer la limite de l'axe des x à 15

# Sauvegarder le graphique
png("distribution-sommeil.png", width = 800, height = 600)
dev.off()

# Le warning est normal : on a cut l'abscisse x aux valeurs > 15

Sur ce second graphique, on s'intéresse à la probabilité de faire une attaque cardiaque selon le nombre d'heures de sommeil. On observe qu'il y a un peu plus de personnes dormant plus de 7h ou moins de 6h parmi celles ayant fait une attaque cardiaque que parmi celles n'en ayant pas fait. Les heures de sommeil et la probabilité de faire une attaque cardiaque semblent donc être corrélés. 

In [ ]:
#Création du graphique représentant la Répartition des catégories d'IMC chez les personnes ayant eu une attaque cardiaque
# Calcul des pourcentages pour chaque catégorie de BMI
Souspoids <- sum(data$BMI < 18.5 & data$HadHeartAttack == 1) / sum(data$BMI < 18.5) * 100
Normal <- sum(data$BMI >= 18.5 & data$BMI < 25 & data$HadHeartAttack == 1) / sum(data$BMI >= 18.5 & data$BMI < 25) * 100
Surpoids <- sum(data$BMI >= 25 & data$BMI < 30 & data$HadHeartAttack == 1) / sum(data$BMI >= 25 & data$BMI < 30) * 100
Obese <- sum(data$BMI >=30 & data$HadHeartAttack == 1) / sum(data$BMI >=30) * 100

# Création des étiquettes pour les portions de camembert avec pourcentage
labels <- paste(c("Souspoids", "Normal", "Surpoids", "Obèse"), " (", round(c(Souspoids, Normal, Surpoids, Obese), 1), "%)", sep = "")

# Création du diagramme en camembert
pie(c(Souspoids, Normal, Surpoids, Obese), labels = labels, main = "Répartition des catégories d'IMC\nchez les personnes ayant eu une attaque cardiaque")

png("IMC.png", width = 800, height = 600)
dev.off()


Ici, on s'intéresse à l'IMC des personnes qui ont fait une attaque cardiaque. Il faut savoir que l'étude a été faite aux Etats-Unis, où l'IMC moyen est de 28,5 (Surpoids). On observe une légère relation entre l'IMC et la probabilité de faire une attaque cardiaque. En effet, être en surpoids ou obèse fait passer la probabilité au dessus de la moyenne calculée. Etre de corpulence normale en revanche, réduit ce risque. Néanmoins, on observe que l'influence n'est pas si élevée que ça. 

In [ ]:
#Création du graphique représentant la Probabilité de faire une attaque cardiaque sachant un état de santé donné
p1 <- sum(data$GeneralHealth_numerique == 1 & data$HadHeartAttack == 1) / sum(data$GeneralHealth_numerique == 1) * 100
p2 <- sum(data$GeneralHealth_numerique == 2 & data$HadHeartAttack == 1) / sum(data$GeneralHealth_numerique == 2) * 100
p3 <- sum(data$GeneralHealth_numerique == 3 & data$HadHeartAttack == 1) / sum(data$GeneralHealth_numerique == 3) * 100
p4 <- sum(data$GeneralHealth_numerique == 4 & data$HadHeartAttack == 1) / sum(data$GeneralHealth_numerique == 4) * 100
p5 <- sum(data$GeneralHealth_numerique == 5 & data$HadHeartAttack == 1) / sum(data$GeneralHealth_numerique == 5) * 100



# Créer un dataframe avec les données
health_data <- data.frame(
  Niveau_Sante = factor(1:5, labels = c("Poor", "Fair", "Good", "Very good", "Excellent")),
  Probabilite_CC = c(p1, p2, p3, p4, p5),
  Pourcentage = c(p1, p2, p3, p4, p5)
)

# Créer le graphique
ggplot(health_data, aes(x = Niveau_Sante, y = Probabilite_CC, label = paste0(round(Pourcentage), "%"))) +
  geom_line(aes(group = 1), color = "blue") +
  geom_point(color = "blue", size = 3) +
  geom_text(vjust = -0.5, color = "black") +
  labs(x = "Niveau de santé général", y = "Probabilité d'attaque cardiaque (%)") +
  ggtitle("Probabilité de faire une attaque cardiaque sachant un état de santé donné") +
  theme_minimal()


png("Etat.png", width = 800, height = 600)
dev.off()

Sur ce graphique, On observe que plus l'état de santé général de la santé de la personne décroît, plus la probabilité de faire une attaque cardiaque est élevée. On atteint jusqu'à 22% pour une personne d'un état de santé "Poor" ce qui est environ 4 fois plus élevé que la moyenne ! Au contraire, avoir un état de santé excellent permet de diviser ce risque par 4. 

In [ ]:
data$AgeInf <- as.integer(data$AgeInf)
#Création du graphique représentant la Probabilité de faire une attaque cardiaque en fonction de l'âge et du sexe de la personne
different_age_inf <- unique(data$AgeInf)
different_age_inf <- sort(different_age_inf, decreasing = FALSE)
n1 <- length(different_age_inf)
nombre_hommes_par_categorie_1 <- numeric(length = n1)
nombre_femmes_par_categorie_1 <- numeric(length = n1)
nombre_hommes_par_categorie_malade_1 <- numeric(length = n1)
nombre_femmes_par_categorie_malade_1 <- numeric(length = n1)
proba_hommes_par_categorie_malade_1 <- numeric(length = n1)
proba_femmes_par_categorie_malade_1 <- numeric(length = n1)

for (i in 1:n1) {
  nombre_hommes_par_categorie_1[i] <- sum((data$Sex == 0) & (data$AgeInf == different_age_inf[i]))
  nombre_femmes_par_categorie_1[i] <- sum((data$Sex == 1) & (data$AgeInf == different_age_inf[i]))
  nombre_hommes_par_categorie_malade_1[i] <- sum((data$Sex == 0) & (data$AgeInf == different_age_inf[i]) & data$HadHeartAttack == 1)
  nombre_femmes_par_categorie_malade_1[i] <- sum((data$Sex == 1) & (data$AgeInf == different_age_inf[i]) & data$HadHeartAttack == 1)
  proba_hommes_par_categorie_malade_1[i] <- (nombre_hommes_par_categorie_malade_1[i] / nombre_hommes_par_categorie_1[i]) * 100
  proba_femmes_par_categorie_malade_1[i] <- (nombre_femmes_par_categorie_malade_1[i] / nombre_femmes_par_categorie_1[i]) * 100
}

data_table <- data.frame(
  x = different_age_inf,
  y = proba_hommes_par_categorie_malade_1,
  z = proba_femmes_par_categorie_malade_1
)

# Créer un graphique avec une légende
ggplot() +
  geom_line(data = data_table, aes(y = y, x = x, color = "Homme"), linetype = "solid", linewidth = 1) +
  geom_line(data = data_table, aes(y = z, x = x, color = "Femme"), linetype = "solid", linewidth = 1) +
  scale_color_manual(name = "Sexe", values = c("Homme" = "blue", "Femme" = "red")) +
  labs(title = "Probabilité de faire une attaque cardiaque en fonction de l'âge et du sexe de la personne", x = "Age (en années)", y = "Risque (en %)") +
  theme_minimal()

png("Vie.png",width=800,height=600)
dev.off()

Ici, on compare les deux indices les plus élémentaires : l'âge de la personne et son sexe. On observe assez logiquement que le risque augmente en fonction de l'âge et qu'une femme a moins de risque de faire une attaque cardiaque qu'un homme. A 80 ans, ce risque est de 20% et deux fois moins pour une femme, on observe bien cette corrélation. 

## Deuxième partie (rendu 2) : Analyse plus technique

In [ ]:
# On retire les variables qui ne sont plus utiles : on décide d'éliminer RaceEthnicityCategory 
#Les trois autres variables sont remplacées par AgeInf et GeneralHealth_numerique
data <- subset(data, select = -c(AgeSup,GeneralHealth,RaceEthnicityCategory,AgeCategory))

Il faut changer les types de nos variables car elles sont pour la plupart des chr et on veut des int/dbl.

In [ ]:
#Changement de types
for (col in names(data)) {
  if (col != "BMI") {
    data[[col]] <- as.integer(data[[col]])
  } else {
    data[[col]] <- as.double(data[[col]])
  }
} 
head(data)

### **Régression logistique**

Nous avons décidé de commencer l'analyse du dataset par une régression logistique car la plupart de nos variables sont binaires. Cela nous permet de faire une prédiction de la probabilité de faire une attaque cardiaque en fonction de toutes les autres variables et d'étudier l'influence des variables sur cette probabilité. 

On vérifie que les types sont corrects. 

In [ ]:
#Creation d'espace d'entrainement et de test
 set.seed(123)
 train_index <- sample(nrow(data), floor(0.8*nrow(data)), replace = FALSE)
 train_data <- data[train_index, ]
 test_data <- data[-train_index, ]


In [ ]:
#Création du modèle de régression logistique
#On entraine notre modèle avec les données du subdataset d'entrainement
model <- glm(HadHeartAttack ~ ., data = train_data, family = binomial)
summary(model)

Par la suite, on s'intéressera aux coefficients (estimate) et aux p-values (Pr(>|z|) afin de comparer l'influence des variables et leurs pertinences.

Pour déterminer l'impact d'une variable dans un modèle on peut calculer son odds ratio et étudier sa position:
- Si odds ratio<1 :  agit avec une tendance à la diminution
- Si odds ratio > 1: agit avec une tendance à l'augmentation
- Si odds ratio=1: action limité

In [ ]:
odds_ratio<-exp(cbind(coef(model), confint(model)))
odds_ratio
# Affiche les odds ratio et leurs intervalls de confiance respectif pour chaque paramètre du modèle
# Attention un peu long à afficher, mettre en commentaire ou en markdown si vous êtes pressé

On peut tirer de ce tableau quelques enseignments :
-  HadStroke,HadDiabetes,SmokersStatus,DifficultyWalking augmente le plus le risque en augmentant
- Au contraire Sex, GénéralHealth_numérique et AlcoholDrinkers( chelou celui la ) semblent diminuer le risque lorsqu'ils augmentent

In [ ]:
#Le modèle entraîné  prédit maintenant les valeurs d'HadHeartAttack de notre espace de test
# On le compare aux vraies valeurs du subdataset de test
y_pred <- predict(model, newdata = test_data, type = "response")



In [ ]:
# Plot Predicted data and original data points
ggplot(test_data, aes(x = y_pred, y = test_data$HadHeartAttack)) +
  geom_point() +
  geom_smooth(method = "glm", aes(y = test_data$HadHeartAttack), method.args = list(family = "binomial"), color = "green", se = FALSE) +
  labs(x = "Predicted Probability", y = "HadHeartAttack")

Ce premier graphique représente les valeurs de HadHeartAttack en fonction de leur probabilité calculée ( la valeur prédite ). On observe que plus la probabilité prédite est élevée, plus l'on se rapproche de la valeur 1 pour HadHeartAttack ce qui est logique : plus la probabilité est élevé, plus l'on a de chances de faire une attaque cardiaque. 

In [ ]:
# Créer une fonction pour calculer la GBE
calculate_GDE <- function(threshold,prediction,test_data) {
  # Convertir les prédictions en classes binaires en utilisant le seuil
  predicted <- ifelse(prediction > threshold, 1, 0)
  if (sum(predicted)== 0 ){
     return (1.0)  
   }
  # Créer la matrice de confusion
  confusion_matrix <- table(Predicted = predicted, Actual = test_data$HadHeartAttack)
  
  # Calculer les valeurs FP et FN
  FP <- confusion_matrix[2, 1]
  FN <- confusion_matrix[1, 2]
  
  # Calculer le nombre total d'observations
  total_observations <- sum(confusion_matrix)
  
  # Calculer la GBE
  GDE <- (FP + FN) / total_observations
  
  return(GDE)
}


On crée une fonction qui calcule la GBE et permets de trouver le seuil optimal, c'est à dire l'endroit où on doit découper la probabilité en deux pour se ramener aux valeurs 0 ( pas de heart attack ) et 1 ( heart attack )

In [ ]:
thresholds <- seq(0.1, 1, by = 0.01)  # Tester différents seuils de 0 à 1 avec un pas de 0.01
GBEs <- sapply(thresholds, function(threshold) calculate_GDE(threshold,y_pred,test_data))

optimal_threshold <- thresholds[which.min(GBEs)]
min_GDE <- min(GBEs)

 ##Afficher le seuil optimal et la GBE minimale
cat("Seuil optimal :", optimal_threshold, "\n")
cat("GBE minimale :", min_GDE, "\n")

In [ ]:
#Le modèle entraîner  prédit maintenant les valeurs d'HadHeartAttack de notre espace de test
# On le compare aux vraies valeurs du subdataset de test
table(Predicted = ifelse(y_pred> optimal_threshold, 1, 0), Actual =test_data$HadHeartAttack )

In [ ]:
dered<-table(Predicted = ifelse(y_pred> 0.5, 1, 0), Actual =test_data$HadHeartAttack )
sensitivity<- (dered[1,2]/(dered[1,1]+dered[1,2]))
specificity<- ((dered[1,2])/(dered[2,1]+dered[2,2]))
accuracy<-(dered[1,1]+ dered[2,2])/(dered[1,1]+dered[1,2]+dered[2,1]+dered[2,2])
precision<- (dered[2,2]/(dered[1,2]+dered[2,2]))
NPV<-(dered[1,1]/(dered[1,2]+dered[1,1]))

print(paste("The sensitivity score is: ------>>  :", sensitivity))
print(paste("The specificity score is: ------>>  :", specificity))
print(paste("The accuraccy score is: ------>>  :", accuracy))
print(paste("The NPV score is: ------>>  :", NPV))
print(paste("The precision score is: ------>>  :", precision))

**Sensibilité :**

    La sensibilité mesure la capacité du modèle à détecter les cas positifs réels.
    Elle est calculée en divisant le nombre de vrais positifs par la somme des vrais positifs et des faux négatifs.

**Spécificité :**

    La spécificité mesure la capacité du modèle à détecter les cas négatifs réels.
    Elle est calculée en divisant le nombre de vrais négatifs par la somme des vrais négatifs et des faux positifs.

**Exactitude (Accuracy) :**

    L'exactitude mesure la proportion de prédictions correctes parmi toutes les prédictions.
    Elle est calculée en divisant le nombre de prédictions correctes par le nombre total de prédictions.

**Valeur prédictive négative (Negative Predictive Value) :**

    La valeur prédictive négative mesure la proportion de vrais négatifs parmi toutes les prédictions négatives.
    Elle est calculée en divisant le nombre de vrais négatifs par la somme des vrais négatifs et des faux négatifs.
    
**Précision (Precision) :**

    La précision mesure la proportion de vrais positifs parmi toutes les prédictions positives.
    Elle est calculée en divisant le nombre de vrais positifs par la somme des vrais positifs et des faux positifs.



In [ ]:
# Obtenez les coefficients estimés de votre modèle
coefficients <- coef(model)
# Obtenez les p-values associées à chaque coefficient
p_values <- summary(model)$coefficients[, 4]

# Créez un data frame pour les coefficients et les p-values
coef_data <- data.frame(
  Variable = names(coefficients),
  Coefficient = coefficients,
  P_Value = p_values
)
png("mon_plot.png")
# Tracez les coefficients
ggplot(coef_data, aes(x = reorder(Variable, Coefficient), y = Coefficient, fill = P_Value < 0.05)) +
  geom_bar(stat = "identity") +
  geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
  coord_flip() +
  labs(title = "Coefficients of Logistic Regression Model", x = "Variable", y = "Coefficient") +
  theme_minimal()
dev.off()
# Tracez les p-values significatives
ggplot(coef_data, aes(x = reorder(Variable, P_Value), y = -log10(P_Value), fill = P_Value < 0.05)) +
  geom_bar(stat = "identity") +
  geom_hline(yintercept = -log10(0.05), linetype = "dashed", color = "red") +
  coord_flip() +
  labs(title = "Significant P-Values of Logistic Regression Model", x = "Variable", y = "-log10(P-Value)") +
  theme_minimal()


On affiche la pente (coefficients) pour chaque variable afin de voir lesquelles ont le plus d'influence sur HadHeartAttack. Les coefficients négatifs correspondent à une influence négative ( diminue le risque ) et les coefficients positifs correspondent à une influence positive ( augmente le risque ). 

Les 3 variables "False" sont les variables exclus, car leur p value est > 0.05 donc elles ne sont pas assez statistiquement significatives pour être prises en compte dans le modèle. 

On affiche également les p values. 

In [ ]:
table(Predicted = ifelse(y_pred> optimal_threshold, 1, 0), Actual =test_data$HadHeartAttack )
cm1 <- table(Predicted = ifelse(y_pred> optimal_threshold, 1, 0), Actual =test_data$HadHeartAttack )


fourfoldplot(cm1, color = c("cyan", "pink"),
             conf.level = 0, margin = 1, main = "Confusion Matrix")



On affiche la matrice de confusion  : 

     Une matrice de confusion est un tableau qui résume les performances d'un modèle de classification en comparant les prédictions du modèle avec les vraies valeurs des données. Elle permet d'évaluer les erreurs de prédiction du modèle en comptant le nombre de vrais positifs, vrais négatifs, faux positifs et faux négatifs.

In [ ]:



# Tracer chaque graphique individuellement
plot(ggeffect(model, "Sex"))


plot(ggeffect(model, "PhysicalActivities"))


plot(ggeffect(model, "SleepHours"))


plot(ggeffect(model, "HadStroke"))


plot(ggeffect(model, "HadAsthma"))


plot(ggeffect(model, "HadSkinCancer"))


plot(ggeffect(model, "HadDiabetes"))


plot(ggeffect(model, "DifficultyWalking"))


plot(ggeffect(model, "SmokerStatus"))


plot(ggeffect(model, "BMI"))


plot(ggeffect(model, "AlcoholDrinkers"))


plot(ggeffect(model, "GeneralHealth_numerique"))


plot(ggeffect(model, "AgeInf"))


On affiche les plot ggeffect pour chaque variable : 

    ggeffect affiche les effets marginaux d'une variable indépendante sur la variable dépendante dans un modèle. 
    Cela permet d'avoir une preuve plus visuelle de l'influence de chaque variable sur HadHeartAttack. 

### Deuxième modèle étudié: Décision Tree


On décide d'effectuer l'étude sur plusieurs datasets différents pour évaluer l'impact de différents paramètres sur nos résultats. 
L'étude de l'influence de chaque variable sur le résultat étant assez complexe avec un decision tree, nous avons décidé de créer plusieurs decision tree. Un premier contenant toutes les variables ( qui servira de comparaison avec la régression logistique et la partie suivante ) et d'autres decision tree auxquels on retire une variable différente à chaque fois. L'idée étant de voir comment cela influe le résultat et donc de déterminer l'influence des variables. 

In [ ]:
# On crée les dataset d'entraînements et de test

train_databis <- subset(train_data, select = -c(PhysicalActivities,DifficultyWalking,SmokerStatus,AlcoholDrinkers))
num_copies <- 10

# Création des copies avec des noms numérotés
for (i in 2:num_copies) {
  assign(paste0("train_data", i), train_databis)
}

test_databis <- subset(test_data, select = -c(PhysicalActivities,DifficultyWalking,SmokerStatus,AlcoholDrinkers))


# Création des copies avec des noms numérotés
for (i in 2:num_copies) {
  assign(paste0("test_data", i), test_databis)
}

train_data2 <- subset(train_data2, select = -c(Sex))
train_data3 <- subset(train_data3, select = -c(SleepHours))
train_data4 <- subset(train_data4, select = -c(HadStroke))
train_data5 <- subset(train_data5, select = -c(HadAsthma))
train_data6 <- subset(train_data6, select = -c(HadSkinCancer))
train_data7 <- subset(train_data7, select = -c(HadDiabetes))
train_data8 <- subset(train_data8, select = -c(BMI))
train_data9 <- subset(train_data9, select = -c(GeneralHealth_numerique))
train_data10 <- subset(train_data10, select = -c(AgeInf))

test_data2 <- subset(test_data2, select = -c(Sex))
test_data3 <- subset(test_data3, select = -c(SleepHours))
test_data4 <- subset(test_data4, select = -c(HadStroke))
test_data5 <- subset(test_data5, select = -c(HadAsthma))
test_data6 <- subset(test_data6, select = -c(HadSkinCancer))
test_data7 <- subset(test_data7, select = -c(HadDiabetes))
test_data8 <- subset(test_data8, select = -c(BMI))
test_data9 <- subset(test_data9, select = -c(GeneralHealth_numerique))
test_data10 <- subset(test_data10, select = -c(AgeInf))

In [ ]:
#decision trees
decision_tree2 <- ctree(HadHeartAttack ~ ., data = train_data2)
decision_tree3 <- ctree(HadHeartAttack ~ ., data = train_data3)
decision_tree4 <- ctree(HadHeartAttack ~ ., data = train_data4)
decision_tree5 <- ctree(HadHeartAttack ~ ., data = train_data5)
decision_tree6 <- ctree(HadHeartAttack ~ ., data = train_data6)
decision_tree7 <- ctree(HadHeartAttack ~ ., data = train_data7)
decision_tree8 <- ctree(HadHeartAttack ~ ., data = train_data8)
decision_tree9 <- ctree(HadHeartAttack ~ ., data = train_data9)
decision_tree10 <- ctree(HadHeartAttack ~ ., data = train_data10)
decision_tree11<- ctree(HadHeartAttack ~ ., data = train_databis)



In [ ]:
# Utiliser le modèle pour prédire les résultats sur les données de test
predictions1 <- predict(decision_tree2, newdata = test_data2, type = "response")
predictions2 <- predict(decision_tree3, newdata = test_data3, type = "response")
predictions3 <- predict(decision_tree4, newdata = test_data4, type = "response")
predictions4 <- predict(decision_tree5, newdata = test_data5, type = "response")
predictions5 <- predict(decision_tree6, newdata = test_data6, type = "response")
predictions6 <- predict(decision_tree7, newdata = test_data7, type = "response")
predictions7 <- predict(decision_tree8, newdata = test_data8, type = "response")
predictions8 <- predict(decision_tree9, newdata = test_data9, type = "response")
predictions9 <- predict(decision_tree10, newdata = test_data10, type = "response")
predictions10 <- predict(decision_tree11, newdata = test_databis, type = "response")


In [ ]:
# Trouver le seuil qui minimise la GBE
  # Calculer la GBE pour chaque seuil
# Trouver le seuil qui minimise la GBE
  # Calculer la GBE pour chaque seuil
GBEs1 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions1,test_data2))
optimal_threshold1 <- thresholds[which.min(GBEs1)]
min_GDE1 <- min(GBEs1)
GBEs2 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions2,test_data3))
optimal_threshold2 <- thresholds[which.min(GBEs2)]
min_GDE2 <- min(GBEs2)
GBEs3 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions3,test_data4))
optimal_threshold3 <- thresholds[which.min(GBEs3)]
min_GDE3 <- min(GBEs3)
GBEs4 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions4,test_data5))
optimal_threshold4 <- thresholds[which.min(GBEs4)]
min_GDE4 <- min(GBEs4)          
GBEs5 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions5,test_data6))
optimal_threshold5 <- thresholds[which.min(GBEs5)]
min_GDE5 <- min(GBEs5)      
GBEs6 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions6,test_data7))
optimal_threshold6 <- thresholds[which.min(GBEs6)]
min_GDE6 <- min(GBEs6)     
GBEs7 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions7,test_data8))
optimal_threshold7 <- thresholds[which.min(GBEs7)]
min_GDE7 <- min(GBEs7)      
GBEs8 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions8,test_data9))
optimal_threshold8 <- thresholds[which.min(GBEs8)]
min_GDE8 <- min(GBEs8)  
GBEs9 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions9,test_data10))
optimal_threshold9 <- thresholds[which.min(GBEs9)]
min_GDE9 <- min(GBEs9)      
GBEs10 <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predictions10,test_databis))
optimal_threshold10 <- thresholds[which.min(GBEs10)]
min_GDE10 <- min(GBEs10)

In [ ]:

 #Afficher le seuil optimal et la GBE minimale
cat("Seuil optimal 1:", optimal_threshold1, "\n")
cat("GBE minimale 1:", min_GDE1, "\n")
cat("\n")
cat("Seuil optimal 2:", optimal_threshold2, "\n")
cat("GBE minimale 2:", min_GDE2, "\n")
cat("\n")
cat("Seuil optimal 3:", optimal_threshold3, "\n")
cat("GBE minimale 3:", min_GDE3, "\n")
cat("\n")
cat("Seuil optimal 4:", optimal_threshold4, "\n")
cat("GBE minimale 4:", min_GDE4, "\n")
cat("\n")
cat("Seuil optimal 5:", optimal_threshold5, "\n")
cat("GBE minimale 5:", min_GDE5, "\n")
cat("\n")

#Afficher le seuil optimal et la GBE minimale
cat("Seuil optimal 6:", optimal_threshold6, "\n")
cat("GBE minimale 6:", min_GDE6, "\n")
cat("\n")
cat("Seuil optimal 7:", optimal_threshold7, "\n")
cat("GBE minimale 7:", min_GDE7, "\n")
cat("\n")
cat("Seuil optimal 8:", optimal_threshold8, "\n")
cat("GBE minimale 8:", min_GDE8, "\n")
cat("\n")
cat("Seuil optimal 9:", optimal_threshold9, "\n")
cat("GBE minimale 9:", min_GDE9, "\n")
cat("\n")
cat("Seuil optimal 10:",optimal_threshold10, "\n")
cat("GBE minimale 10:", min_GDE10, "\n")
cat("\n")


In [ ]:
# Nombre de prédictions
num_predictions <- 10

# Boucle pour traiter chaque prédiction
for (i in 1:num_predictions) {
  current_prediction <- get(paste0("predictions", i))  # Sélectionner la prédiction actuelle
  
  # Modifier la prédiction actuelle en fonction de la condition
  for (j in 1:length(current_prediction)) {
    if (current_prediction[j] <get(paste0("optimal_threshold", i))) {
      current_prediction[j] <- 0
    } else {
      current_prediction[j] <- 1
    }
  }
  
  # Mettre à jour la prédiction dans l'environnement
  assign(paste0("predictions", i), current_prediction)
}

In [ ]:
print("Accuracies : ")
accuracy1 <- mean(predictions1 == test_data2$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy1, "\n")
accuracy2 <- mean(predictions2 == test_data3$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy2, "\n")
accuracy3 <- mean(predictions3 == test_data4$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy3, "\n")
accuracy4 <- mean(predictions4 == test_data5$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy4, "\n")
accuracy5 <- mean(predictions5 == test_data6$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy5, "\n")
accuracy6 <- mean(predictions6 == test_data7$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy6, "\n")
accuracy7 <- mean(predictions7 == test_data8$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy7, "\n")
accuracy8 <- mean(predictions8 == test_data9$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy8, "\n")
accuracy9 <- mean(predictions9 == test_data10$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy9, "\n")
accuracy10 <- mean(predictions10 == test_databis$HadHeartAttack)
cat("Exactitude du modèle sur les données de test:", accuracy10, "\n")

On observe ici les différences d'exactitude pour chaque variable.

In [ ]:
# Créer la table de contingence
  table_results <- table(Predicted = ifelse(predictions10 > optimal_threshold10, 1, 0),
                         Actual = test_databis$HadHeartAttack)
  

  
  # Afficher la matrice de confusion
  fourfoldplot(table_results, color = c("cyan", "pink"),
               conf.level = 0, margin = 1)
# Définir le nombre de prédictions
num_predictions <- 9

# Créer un mfrow pour afficher les graphiques
par(mfrow = c(2, 5))

# Boucle sur les prédictions
for (i in 1:num_predictions) {
  current_prediction <- get(paste0("predictions", i))
  

  
  # Créer la table de contingence
  table_results <- table(Predicted = ifelse(current_prediction > get(paste0("optimal_threshold", i)), 1, 0),
                         Actual = get(paste0("test_data", (i+1)))$HadHeartAttack)
  

  
  # Afficher la matrice de confusion
  fourfoldplot(table_results, color = c("cyan", "pink"),
               conf.level = 0, margin = 1)
  

}


On crée les matrices de confusion pour chaque cas. 

In [ ]:
dered<-table(Predicted = ifelse(predictions10> optimal_threshold10, 1, 0), Actual =test_databis$HadHeartAttack )
sensitivity<- (dered[1,2]/(dered[1,1]+dered[1,2]))
specificity<- ((dered[1,2])/(dered[2,1]+dered[2,2]))
accuracy<-(dered[1,1]+ dered[2,2])/(dered[1,1]+dered[1,2]+dered[2,1]+dered[2,2])
precision<- (dered[2,2]/(dered[1,2]+dered[2,2]))
NPV<-(dered[1,1]/(dered[1,2]+dered[1,1]))

print(paste("The sensitivity score is: ------>>  :", sensitivity))
print(paste("The specificity score is: ------>>  :", specificity))
print(paste("The accuraccy score is: ------>>  :", accuracy))
print(paste("The NPV score is: ------>>  :", NPV))
print(paste("The precision score is: ------>>  :", precision))

On calcule la sensitivity, specificity, accuracy, precision et NPV pour le cas avec toutes les variables. 

### **Random forest**

Nous avons choisi de faire une random forest car elle utilise les arbres de décision, elle permet d'améliorer la précision obtenu avec le decision tree car elle en utilise plusieurs. 

Par soucis technique, nous devons réduire la taille de l'échantillon choisi, sinon le code est trpo long à l'exéution. 

In [ ]:
data2<-data[1:650,]

head(data2)

In [ ]:
#Creation d'espace d'entrainement et de test
#Creation d'espace d'entrainement et de test
 set.seed(123)
 rftrain_index <- sample(nrow(data2), floor(0.8*nrow(data2)), replace = FALSE)
 rftrain_data <- data[rftrain_index, ]
 rftest_data <- data[-rftrain_index, ]

In [ ]:
set.seed(123)
ml_rf = randomForest(formula = HadHeartAttack ~ ., 
                         data =rftrain_data,
                         mtry = sqrt(ncol(rftrain_data)-1), # minus the response
                         importance = T, # to estimate predictors importance
                         ntree = 500) # 500 by default
ml_rf
#As before we plot the measure for the importance of the regressors:
varImpPlot(ml_rf)

On affiche le résultat de la random forest et on affiche varImpPlot qui plot l'importance des variables mesurée par la diminution de l'erreur de prédiction lorsqu'une variable est incluse dans les arbres de la forêt. 

In [ ]:
predrf <- predict(ml_rf, newdata = rftest_data, type = "response")

In [ ]:
GBEsrf <- sapply(thresholds, function(threshold) calculate_GDE(threshold,predrf,rftest_data))
optimal_thresholdrf <- thresholds[which.min(GBEsrf)]
min_GDErf <- min(GBEsrf)

In [ ]:
cat("Seuil optimal rf:",optimal_thresholdrf, "\n")
cat("GBE minimale rf:", min_GDErf, "\n")
cat("\n")

In [ ]:
table(Predicted = ifelse(predrf> optimal_thresholdrf, 1, 0), Actual =rftest_data$HadHeartAttack )
cm1 <- table(Predicted = ifelse(predrf> optimal_thresholdrf, 1, 0), Actual =rftest_data$HadHeartAttack )


fourfoldplot(cm1, color = c("cyan", "pink"),
             conf.level = 0, margin = 1, main = "Confusion Matrix")

On peut voir ici les limites du Bayes Test et de la minimisation du GDE. Ici la meilleure solution au sens de Bayes nous donne un seuil  pour le  modèle qui ne prédit que des valeurs nulles et qui est donc obsolète.

In [ ]:
table(Predicted = ifelse(predrf> 0.2, 1, 0), Actual =rftest_data$HadHeartAttack )
cm1 <- table(Predicted = ifelse(predrf> 0.2, 1, 0), Actual =rftest_data$HadHeartAttack )


fourfoldplot(cm1, color = c("cyan", "pink"),
             conf.level = 0, margin = 1, main = "Confusion Matrix")

In [ ]:
dered<-table(Predicted = ifelse(predrf> 0.5, 1, 0), Actual =rftest_data$HadHeartAttack )
sensitivity<- (dered[1,2]/(dered[1,1]+dered[1,2]))
specificity<- ((dered[1,2])/(dered[2,1]+dered[2,2]))
accuracy<-(dered[1,1]+ dered[2,2])/(dered[1,1]+dered[1,2]+dered[2,1]+dered[2,2])
precision<- (dered[2,2]/(dered[1,2]+dered[2,2]))
NPV<-(dered[1,1]/(dered[1,2]+dered[1,1]))

print(paste("The sensitivity score is: ------>>  :", sensitivity))
print(paste("The specificity score is: ------>>  :", specificity))
print(paste("The accuraccy score is: ------>>  :", accuracy))
print(paste("The NPV score is: ------>>  :", NPV))
print(paste("The precision score is: ------>>  :", precision))

### Cross validation 

La Cross Validation est une méthode de rééchantillonnage statistique utilisée pour évaluer les performances du modèle d'apprentissage automatique sur des données qu'il ne voit pas aussi objectivement et précisément que possible.

Ici nous effectuons une méthode K-Fold qui appliquent différentes étapes.

Elle divise l'espace d'entrainement en n_fold sous6dataset.
Dans notre cas , on prendra n_fold=9.

On nomme ensuite un sous data_set de validation , les huits autres serviront d'entrainement. On établit alors le modèle avec ces dernieres huits parties et on l'évalue avec le sous-dataset de validation. On calcule alors l'accuracy.

On procède de même en nommant un par un les divers sous_datasets en jeu de validation. 

On enregistre à chaque pas l'accuracy pour pouvoir comparer a la fin 

In [ ]:
# Cross Validation

ctrl <- trainControl(method = "cv", number = 9 )  # k = 9, vous pouvez choisir un autre nombre selon vos besoins
ctrl

#### Cross validation pour la régression

Poue effectuer une classification et ainsi obtenir une accuracy, on est forcé de transformé notre variable de réponse HadHeartAttack en factor. Il en sera de même pour les autres modèles 

In [ ]:
train_data$HadHeartAttack <-factor(train_data$HadHeartAttack, levels = c(0, 1))
results <- train(HadHeartAttack ~ ., data = train_data, method = "glm", trControl = ctrl,metric= "Accuracy")  # Remplacez "target" par le nom de votre variable cible
print(results)


#### Cross Validation pour Random Forest

In [ ]:
rftrain_data$HadHeartAttack <-factor(rftrain_data$HadHeartAttack, levels = c(0, 1))
results3 <- train(HadHeartAttack ~ ., data = rftrain_data, method = "rf", trControl = ctrl,metric= "Accuracy")  # Remplacez "target" par le nom de votre variable cible
print(results3)

#### Cross Validation pour décision tree

In [ ]:
train_databis$HadHeartAttack <-factor(train_databis$HadHeartAttack, levels = c(0, 1))
results2 <- train(HadHeartAttack ~ ., data = train_databis, method = "ctree", trControl = ctrl,metric= "Accuracy")  # Remplacez "target" par le nom de votre variable cible
print(results2)


On remarque que l'accuracy calculé du random forest a diminué par rapport au calcul précédemment effectué (87<94). Cela est surement la conséquence d'un suraprentissage du modèle.

In [ ]:
rftrain_data$HadHeartAttack <-as.integer(rftrain_data$HadHeartAttack)
train_data$HadHeartAttack <-as.integer(train_data$HadHeartAttack)
train_databis$HadHeartAttack <-as.integer(train_databis$HadHeartAttack)
rftest_data$HadHeartAttack <-as.integer(rftest_data$HadHeartAttack)
data$HadHeartAttack <-as.integer(data$HadHeartAttack)